# Chapter 1: Introduction to Qlib and Basic Configuration
# 第一章：Qlib 简介与基础配置

**Author**: Microsoft Qlib Team  
**License**: MIT License  
**Last Updated**: 2025-01-09

---

## 📚 Table of Contents / 目录

1. [What is Qlib? / Qlib是什么？](#what-is-qlib)
2. [Installation / 安装](#installation)
3. [Basic Configuration / 基础配置](#basic-configuration)
4. [Qlib Architecture / Qlib架构](#qlib-architecture)
5. [Key Concepts / 核心概念](#key-concepts)
6. [Setting Up Your First Experiment / 设置第一个实验](#first-experiment)
7. [Common Configuration Patterns / 常用配置模式](#configuration-patterns)
8. [Next Steps / 下一步](#next-steps)

## 1. What is Qlib? / Qlib是什么？ <a id='what-is-qlib'></a>

### English
**Qlib** is an AI-oriented quantitative investment platform developed by Microsoft, designed to facilitate research and implementation of quantitative trading strategies. It provides:

- **Infrastructure** for managing financial data
- **High-performance** data processing and caching
- **Machine Learning** toolkit specifically designed for finance
- **Backtesting** framework for strategy evaluation
- **Workflow Management** for reproducible research

### 中文
**Qlib** 是微软开发的面向AI的量化投资平台，旨在促进量化交易策略的研究和实施。它提供：

- 金融数据管理**基础设施**
- **高性能**数据处理和缓存
- 专为金融设计的**机器学习**工具包
- 策略评估的**回测**框架
- 可重现研究的**工作流管理**

## 2. Installation / 安装 <a id='installation'></a>

### Step 1: Install Qlib / 第一步：安装Qlib

In [60]:
# Install Qlib from PyPI / 从PyPI安装Qlib
# Uncomment the following line to install / 取消注释以下行进行安装
# !pip install qlib

# Or install from source for latest features / 或从源码安装以获得最新功能
# !git clone https://github.com/microsoft/qlib.git
# !cd qlib && pip install .

# Install additional dependencies / 安装额外依赖
# !pip install lightgbm matplotlib plotly pandas numpy

### Step 2: Verify Installation / 第二步：验证安装

In [61]:
import sys
import warnings
warnings.filterwarnings('ignore')

# Check Python version / 检查Python版本
print(f"Python version: {sys.version}")
print("-" * 50)

# Import and check Qlib / 导入并检查Qlib
try:
    import qlib
    print(f"✅ Qlib version: {qlib.__version__}")
except ImportError:
    print("❌ Qlib not installed. Please install it first.")

# Check other dependencies / 检查其他依赖
dependencies = {
    'pandas': 'pd',
    'numpy': 'np',
    'matplotlib': 'plt',
    'lightgbm': 'lgb',
    'plotly': 'plotly'
}

for package, alias in dependencies.items():
    try:
        module = __import__(package)
        print(f"✅ {package}: {module.__version__}")
    except ImportError:
        print(f"⚠️ {package}: Not installed (optional)")

Python version: 3.12.11 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 12:58:53) [MSC v.1929 64 bit (AMD64)]
--------------------------------------------------
✅ Qlib version: 0.9.7
✅ pandas: 2.3.2
✅ numpy: 2.3.2
✅ matplotlib: 3.10.6
✅ lightgbm: 4.6.0
✅ plotly: 6.3.0


## 3. Basic Configuration / 基础配置 <a id='basic-configuration'></a>

### 3.1 Initialize Qlib / 初始化Qlib

In [62]:
import qlib
from pathlib import Path

# Method 1: Default initialization / 方法1：默认初始化
# This will use the default data directory ~/.qlib/qlib_data/cn_data
# 这将使用默认数据目录 ~/.qlib/qlib_data/cn_data
qlib.init()

print("✅ Qlib initialized with default settings")

[23872:MainThread](2025-09-10 00:51:25,442) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[23872:MainThread](2025-09-10 00:51:25,443) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[23872:MainThread](2025-09-10 00:51:25,444) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/zihao/.qlib/qlib_data/cn_data')}


✅ Qlib initialized with default settings


In [63]:
# Method 2: Custom initialization / 方法2：自定义初始化
# You can specify custom paths and settings / 你可以指定自定义路径和设置

# custom_config = {
#     "provider_uri": "~/.qlib/qlib_data/cn_data",  # Data directory / 数据目录
#     "region": "cn",  # Region: cn (China) or us (United States) / 地区：cn（中国）或us（美国）
#     "redis_host": "127.0.0.1",  # Optional: Redis for caching / 可选：Redis缓存
#     "redis_port": 6379,
#     "kernels": 1,  # Number of kernels for parallel computing / 并行计算的核心数
# }
# qlib.init(**custom_config)

### 3.2 Download Data / 下载数据

Qlib需要历史市场数据才能运行。首次使用时需要下载数据。

In [64]:
from qlib.tests.data import GetData

# Download CN (China A-share) data / 下载中国A股数据
# This will download about 1GB of data / 这将下载约1GB的数据
# Set exists_skip=True to skip if data already exists / 设置exists_skip=True以跳过已存在的数据
GetData().qlib_data(exists_skip=True)

print("✅ Data download complete")

2025-09-10 00:51:26.194 | WARNING  | qlib.tests.data:qlib_data:195 - Data already exists: ~/.qlib/qlib_data/cn_data, the data download will be skipped
	If downloading is required: `exists_skip=False` or `change target_dir`


✅ Data download complete


## 4. Qlib Architecture / Qlib架构 <a id='qlib-architecture'></a>

### Core Components / 核心组件

```
┌─────────────────────────────────────────────────────────────┐
│                     Qlib Architecture                       │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐       │
│  │   Data      │  │   Model     │  │  Portfolio  │       │
│  │   Layer     │→ │   Layer     │→ │  Layer      │       │
│  └─────────────┘  └─────────────┘  └─────────────┘       │
│         ↓                ↓                ↓               │
│  ┌─────────────────────────────────────────────┐         │
│  │           Workflow Management                │         │
│  │         (Experiment Tracking)                │         │
│  └─────────────────────────────────────────────┘         │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

1. **Data Layer / 数据层**
   - Data retrieval and storage / 数据检索和存储
   - Feature engineering / 特征工程
   - Data preprocessing / 数据预处理

2. **Model Layer / 模型层**
   - ML models for prediction / 用于预测的机器学习模型
   - Training and inference / 训练和推理
   - Model management / 模型管理

3. **Portfolio Layer / 组合层**
   - Strategy execution / 策略执行
   - Backtesting / 回测
   - Risk management / 风险管理

4. **Workflow Management / 工作流管理**
   - Experiment tracking / 实验跟踪
   - Result analysis / 结果分析
   - Reproducibility / 可重现性

## 5. Key Concepts / 核心概念 <a id='key-concepts'></a>

In [65]:
# Import commonly used modules / 导入常用模块
from qlib.data import D  # Data API
from qlib.config import C  # Config API
import pandas as pd
import numpy as np

### 5.1 Instruments / 股票标的

In [66]:
# Get instrument list / 获取股票列表

# Static universe (fixed stock pool) / 静态股票池（固定股票池）
instruments = D.instruments(market='csi300') # 沪深300
print(f"CSI300 instruments type: {type(instruments)}")
print(f"Example format: {instruments}\n")

# Dynamic universe (changes over time) / 动态股票池（随时间变化）
# Get CSI300 constituents for a specific date range
# 获取特定日期范围的沪深300成分股
dynamic_instruments = D.list_instruments(
    instruments=D.instruments('csi300'),
    start_time='2020-01-01',
    end_time=pd.Timestamp.now()
)

# 生成每月的首日列表
month_starts = pd.date_range(start='2020-01-01', end=pd.Timestamp.now(), freq='MS')

print("每月成分股数量（沪深300动态股票池）:")
print("="*50)

# 只显示最近12个月的数据作为演示
recent_months = month_starts[-24:]
for idx, month_start in enumerate(recent_months):
    # 计算该月的结束日期
    if idx < len(recent_months) - 1:
        month_end = recent_months[idx + 1] - pd.Timedelta(days=1)
    else:
        month_end = pd.Timestamp.now()
    
    try:
        # 获取该月的成分股
        month_instruments = D.list_instruments(
            instruments=D.instruments('csi300'),
            start_time=month_start.strftime('%Y-%m-%d'),
            end_time=month_end.strftime('%Y-%m-%d')
        )
        print(f"{month_start.strftime('%Y-%m')} 的成分股数量: {len(month_instruments)}")
    except Exception as e:
        print(f"{month_start.strftime('%Y-%m')} 获取失败: {e}")

print("\n注意：沪深300成分股每半年调整一次（6月和12月）")




CSI300 instruments type: <class 'dict'>
Example format: {'market': 'csi300', 'filter_pipe': []}

每月成分股数量（沪深300动态股票池）:
2023-10 的成分股数量: 300
2023-11 的成分股数量: 300
2023-12 的成分股数量: 314
2024-01 的成分股数量: 300
2024-02 的成分股数量: 300
2024-03 的成分股数量: 300
2024-04 的成分股数量: 300
2024-05 的成分股数量: 300
2024-06 的成分股数量: 312
2024-07 的成分股数量: 300
2024-08 的成分股数量: 300
2024-09 的成分股数量: 300
2024-10 的成分股数量: 300
2024-11 的成分股数量: 300
2024-12 的成分股数量: 316
2025-01 的成分股数量: 300
2025-02 的成分股数量: 300
2025-03 的成分股数量: 300
2025-04 的成分股数量: 300
2025-05 的成分股数量: 300
2025-06 的成分股数量: 308
2025-07 的成分股数量: 300
2025-08 的成分股数量: 300
2025-09 的成分股数量: 300

注意：沪深300成分股每半年调整一次（6月和12月）


### 5.2 Calendar / 交易日历

In [67]:
# Get trading calendar / 获取交易日历
# This shows all trading days (excludes weekends and holidays)
# 显示所有交易日（不包括周末和节假日）

calendar = D.calendar(
    start_time='2024-01-01',
    end_time='2024-01-31',
    freq='day'
)

print(f"Trading days in Jan 2024: {len(calendar)} days")
print(f"First 5 trading days:\n{calendar[:5]}")
print(f"\nLast 5 trading days:\n{calendar[-5:]}")

Trading days in Jan 2024: 22 days
First 5 trading days:
[Timestamp('2024-01-02 00:00:00') Timestamp('2024-01-03 00:00:00')
 Timestamp('2024-01-04 00:00:00') Timestamp('2024-01-05 00:00:00')
 Timestamp('2024-01-08 00:00:00')]

Last 5 trading days:
[Timestamp('2024-01-25 00:00:00') Timestamp('2024-01-26 00:00:00')
 Timestamp('2024-01-29 00:00:00') Timestamp('2024-01-30 00:00:00')
 Timestamp('2024-01-31 00:00:00')]


### 5.3 Features / 特征数据

In [68]:
# Basic price and volume features / 基础价格和成交量特征
# $ prefix indicates field name / $前缀表示字段名

basic_features = [
    "$open",   # 开盘价
    "$high",   # 最高价
    "$low",    # 最低价
    "$close",  # 收盘价
    "$volume", # 成交量
    "$factor", # 复权因子
]

# Get data for specific stocks / 获取特定股票的数据
data = D.features(
    instruments=['SH600000', 'SH600519'],  # 浦发银行, 贵州茅台
    fields=basic_features,
    start_time='2024-01-01',
    end_time='2024-01-10',
    freq='day'
)

print("Raw price data / 原始价格数据:")
print(data.head(10))

Raw price data / 原始价格数据:
                            $open       $high        $low      $close  \
instrument datetime                                                     
SH600000   2024-01-02    9.608777    9.637763    9.565298    9.565298   
           2024-01-03    9.550671    9.637627    9.550671    9.623135   
           2024-01-04    9.623202    9.666680    9.492767    9.594216   
           2024-01-05    9.565030    9.796910    9.550538    9.680970   
           2024-01-08    9.680803    9.724279    9.506896    9.550373   
           2024-01-09    9.564799    9.622767    9.477846    9.579291   
           2024-01-10    9.579425    9.608409    9.521455    9.521455   
SH600519   2024-01-02  227.355698  227.778595  222.463898  223.379959   
           2024-01-03  222.862976  224.733521  222.229294  224.571793   
           2024-01-04  224.439117  224.439117  220.452774  221.257462   

                             $volume   $factor  
instrument datetime                             


### 5.4 Expression Engine / 表达式引擎

In [69]:
# Qlib's expression engine allows complex feature calculation
# Qlib的表达式引擎允许复杂的特征计算

# Calculate technical indicators / 计算技术指标
technical_features = [
    "Ref($close, 1)",                    # Yesterday's close / 昨日收盘价
    "$close / Ref($close, 1) - 1",       # Daily return / 日收益率
    "Mean($close, 5)",                   # 5-day moving average / 5日均线
    "Mean($close, 20)",                  # 20-day moving average / 20日均线
    "Std($close, 20)",                   # 20-day volatility / 20日波动率
    "($high - $low) / $close",           # Daily range / 日内振幅
    "Mean($volume, 5) / Mean($volume, 20)",  # Volume ratio / 量比
]

# Feature names for display / 用于显示的特征名称
feature_names = [
    "prev_close",
    "daily_return",
    "ma5",
    "ma20",
    "volatility",
    "daily_range",
    "volume_ratio"
]

# Calculate features / 计算特征
tech_data = D.features(
    instruments=['SH600519'],  # 贵州茅台
    fields=technical_features,
    start_time='2024-01-01',
    end_time='2024-01-31',
    freq='day'
)
tech_data.columns = feature_names

print("Technical indicators / 技术指标:")
print(tech_data.tail(10))
print(f"\nData shape: {tech_data.shape}")

Technical indicators / 技术指标:
                       prev_close  daily_return         ma5        ma20  \
instrument datetime                                                       
SH600519   2024-01-18  209.061249      0.021478  215.036346  220.034515   
           2024-01-19  213.551376      0.014979  214.822647  219.802521   
           2024-01-22  216.750122      0.004282  214.875687  219.612625   
           2024-01-23  217.678253     -0.014007  214.334030  219.261307   
           2024-01-24  214.629150      0.004324  215.633209  218.969666   
           2024-01-25  215.557114      0.007380  216.352524  218.777039   
           2024-01-26  217.147964     -0.000416  216.414032  218.195923   
           2024-01-29  217.057709      0.007744  216.626114  217.692169   
           2024-01-30  218.738647     -0.022424  216.467010  217.214844   
           2024-01-31  213.833649     -0.005015  215.907852  216.624313   

                       volatility  daily_range  volume_ratio  
instrum

## 6. Setting Up Your First Experiment / 设置第一个实验 <a id='first-experiment'></a>

In [70]:
# Define experiment parameters / 定义实验参数
# These will be used throughout the workflow / 这些将在整个工作流程中使用

# Market configuration / 市场配置
MARKET = "csi300"          # Target market / 目标市场
BENCHMARK = "SH000300"     # Benchmark index / 基准指数

# Experiment configuration / 实验配置
EXP_NAME = "tutorial_exp"  # Experiment name / 实验名称

# Time configuration / 时间配置
TRAIN_START = "2008-01-01"  # Training start date / 训练集开始日期
TRAIN_END = "2014-12-31"    # Training end date / 训练集结束日期
VALID_START = "2015-01-01"  # Validation start date / 验证集开始日期
VALID_END = "2016-12-31"    # Validation end date / 验证集结束日期
TEST_START = "2017-01-01"   # Test start date / 测试集开始日期
TEST_END = "2020-08-01"     # Test end date / 测试集结束日期

print("Experiment Configuration / 实验配置:")
print("=" * 50)
print(f"Market: {MARKET}")
print(f"Benchmark: {BENCHMARK}")
print(f"Experiment Name: {EXP_NAME}")
print(f"\nTime Periods / 时间段:")
print(f"  Training:   {TRAIN_START} to {TRAIN_END}")
print(f"  Validation: {VALID_START} to {VALID_END}")
print(f"  Testing:    {TEST_START} to {TEST_END}")

Experiment Configuration / 实验配置:
Market: csi300
Benchmark: SH000300
Experiment Name: tutorial_exp

Time Periods / 时间段:
  Training:   2008-01-01 to 2014-12-31
  Validation: 2015-01-01 to 2016-12-31
  Testing:    2017-01-01 to 2020-08-01


## 7. Common Configuration Patterns / 常用配置模式 <a id='configuration-patterns'></a>

In [71]:
# Configuration dictionary pattern / 配置字典模式
# Qlib uses configuration dictionaries extensively / Qlib广泛使用配置字典

# Example: Model configuration / 示例：模型配置
model_config = {
    "class": "LGBModel",
    "module_path": "qlib.contrib.model.gbdt",
    "kwargs": {
        "loss": "mse",
        "colsample_bytree": 0.8879,
        "learning_rate": 0.0421,
        "subsample": 0.8789,
        "lambda_l1": 205.6999,
        "lambda_l2": 580.9768,
        "max_depth": 8,
        "num_leaves": 210,
        "num_threads": 20,
    }
}

# Example: Data handler configuration / 示例：数据处理器配置
handler_config = {
    "class": "Alpha158",
    "module_path": "qlib.contrib.data.handler",
    "kwargs": {
        "start_time": TRAIN_START,
        "end_time": TEST_END,
        "fit_start_time": TRAIN_START,
        "fit_end_time": TRAIN_END,
        "instruments": MARKET,
    }
}

# Example: Dataset configuration / 示例：数据集配置
dataset_config = {
    "class": "DatasetH",
    "module_path": "qlib.data.dataset",
    "kwargs": {
        "handler": handler_config,
        "segments": {
            "train": (TRAIN_START, TRAIN_END),
            "valid": (VALID_START, VALID_END),
            "test": (TEST_START, TEST_END),
        },
    },
}

print("Configuration examples created successfully!")
print("These configurations will be used in subsequent notebooks.")

Configuration examples created successfully!
These configurations will be used in subsequent notebooks.


## 8. Workflow Management / 工作流管理

In [72]:
# Qlib uses MLflow for experiment tracking / Qlib使用MLflow进行实验跟踪
from qlib.workflow import R

# Start experiment using R.start() / 使用R.start()启动实验
# This is the recommended way to create and manage experiments in Qlib
# 这是Qlib中创建和管理实验的推荐方式

print("Experiment Configuration / 实验配置:")
print("=" * 50)
print(f"Experiment Name: {EXP_NAME}")
print(f"Market: {MARKET}")
print(f"Benchmark: {BENCHMARK}")

# Example of using R.start() context manager
# R.start()上下文管理器的示例
print("\nTo start an experiment, use:")
print("要启动实验，请使用：")
print("""
with R.start(experiment_name=EXP_NAME):
    # Your model training code here
    # 你的模型训练代码
    pass
""")

Experiment Configuration / 实验配置:
Experiment Name: tutorial_exp
Market: csi300
Benchmark: SH000300

To start an experiment, use:
要启动实验，请使用：

with R.start(experiment_name=EXP_NAME):
    # Your model training code here
    # 你的模型训练代码
    pass

Note: R.start() will be used in subsequent notebooks for actual experiments
注意：R.start()将在后续notebook中用于实际实验


## 9. Best Practices / 最佳实践

### 9.1 Memory Management / 内存管理

```python
# Good practice: Load data in chunks / 好习惯：分块加载数据
# Avoid loading entire market data at once / 避免一次加载整个市场数据

# Bad / 不好
# all_data = D.features(D.instruments('all'), fields, '2008-01-01', '2024-12-31')

# Good / 好
# for year in range(2008, 2025):
#     yearly_data = D.features(instruments, fields, f'{year}-01-01', f'{year}-12-31')
#     process(yearly_data)
```

### 9.2 Error Handling / 错误处理

```python
# Always check data availability / 始终检查数据可用性
try:
    data = D.features(['SH600000'], ['$close'], '2024-01-01', '2024-01-31')
    if data.empty:
        print("Warning: No data available")
except Exception as e:
    print(f"Error loading data: {e}")
```

### 9.3 Reproducibility / 可重现性

```python
# Set random seeds for reproducibility / 设置随机种子以确保可重现性
import random
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
```

## 10. Quick Reference / 快速参考

In [73]:
# Common imports / 常用导入
print("""Common Qlib Imports / 常用Qlib导入:
=====================================
import qlib
from qlib.data import D                          # Data API
from qlib.config import C                        # Config API
from qlib.workflow import R                      # Recorder API
from qlib.workflow.exp import Experiment         # Experiment management
from qlib.utils import init_instance_by_config   # Config instantiation
from qlib.data.dataset import DatasetH, TSDatasetH  # Dataset classes
from qlib.data.dataset.handler import DataHandlerLP # Data handler
from qlib.contrib.data.handler import Alpha158   # Pre-built feature set
from qlib.contrib.model.gbdt import LGBModel     # LightGBM model
from qlib.contrib.report import analysis_model, analysis_position
""")

print("\nCommon Operations / 常用操作:")
print("="*40)
print("""# Initialize Qlib / 初始化Qlib
qlib.init()

# Get instruments / 获取股票
instruments = D.instruments('csi300')

# Get calendar / 获取日历
calendar = D.calendar('2024-01-01', '2024-12-31')

# Get features / 获取特征
data = D.features(instruments, fields, start, end)

# Start experiment / 开始实验
with R.start(experiment_name='exp_name'):
    # training code
    pass
""")

Common Qlib Imports / 常用Qlib导入:
import qlib
from qlib.data import D                          # Data API
from qlib.config import C                        # Config API
from qlib.workflow import R                      # Recorder API
from qlib.workflow.exp import Experiment         # Experiment management
from qlib.utils import init_instance_by_config   # Config instantiation
from qlib.data.dataset import DatasetH, TSDatasetH  # Dataset classes
from qlib.data.dataset.handler import DataHandlerLP # Data handler
from qlib.contrib.data.handler import Alpha158   # Pre-built feature set
from qlib.contrib.model.gbdt import LGBModel     # LightGBM model
from qlib.contrib.report import analysis_model, analysis_position


Common Operations / 常用操作:
# Initialize Qlib / 初始化Qlib
qlib.init()

# Get instruments / 获取股票
instruments = D.instruments('csi300')

# Get calendar / 获取日历
calendar = D.calendar('2024-01-01', '2024-12-31')

# Get features / 获取特征
data = D.features(instruments, fields, start, end)

# S

## Next Steps / 下一步 <a id='next-steps'></a>

### 📚 Continue with the following notebooks / 继续学习以下notebook：

1. **[02_data_module.ipynb](./02_data_module.ipynb)**
   - Deep dive into data loading and preprocessing / 深入了解数据加载和预处理
   - Learn about DataLoader, DataHandler, and Datasets / 学习DataLoader、DataHandler和Datasets

2. **[03_model_training.ipynb](./03_model_training.ipynb)**
   - Train your first prediction model / 训练你的第一个预测模型
   - Understand model interfaces and training workflow / 理解模型接口和训练工作流

3. **[04_evaluation_module.ipynb](./04_evaluation_module.ipynb)**
   - Learn backtesting and performance evaluation / 学习回测和性能评估
   - Analyze trading strategies / 分析交易策略

4. **[05_utils_and_helpers.ipynb](./05_utils_and_helpers.ipynb)**
   - Utility functions and advanced features / 工具函数和高级功能
   - Tips and tricks for efficient research / 高效研究的技巧和窍门

### 📖 Additional Resources / 额外资源

- [Qlib Documentation](https://qlib.readthedocs.io/)
- [Qlib GitHub Repository](https://github.com/microsoft/qlib)
- [Qlib Examples](https://github.com/microsoft/qlib/tree/main/examples)
- [Qlib Paper](https://arxiv.org/abs/2009.11189)

## Summary / 总结

In this notebook, we covered / 在本notebook中，我们介绍了：

✅ **Qlib installation and setup** / Qlib安装和设置  
✅ **Basic configuration** / 基础配置  
✅ **Core concepts** (instruments, calendar, features) / 核心概念（股票、日历、特征）  
✅ **Expression engine** for feature calculation / 用于特征计算的表达式引擎  
✅ **Experiment setup** / 实验设置  
✅ **Best practices** / 最佳实践  

You are now ready to proceed with the data module! / 你现在可以继续学习数据模块了！

In [78]:
# Save configuration for use in other notebooks / 保存配置供其他notebook使用
import json

config = {
    "market": MARKET,
    "benchmark": BENCHMARK,
    "exp_name": EXP_NAME,
    "train_start": TRAIN_START,
    "train_end": TRAIN_END,
    "valid_start": VALID_START,
    "valid_end": VALID_END,
    "test_start": TEST_START,
    "test_end": TEST_END,
}

# Save to file / 保存到文件
with open('experiment_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("Configuration saved to experiment_config.json")
print("You can load this in other notebooks using:")
print("""\nwith open('experiment_config.json', 'r') as f:
    config = json.load(f)""")

Configuration saved to experiment_config.json
You can load this in other notebooks using:

with open('experiment_config.json', 'r') as f:
    config = json.load(f)
